In [4]:
from sklearn.neural_network import MLPRegressor
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [5]:
# y -> outlet liquid temperature
# q(t) -> liquid flow rate
# Narx -> yhat(t+1|t) = f(y(t) ... y(t-3) u(t) ... u(t-9))

#read data
data = open('../exchanger/exchanger.dat')
lst = []
for line in data:
    lst += [line.split()]
time = [float(x[0]) for x in lst]
u = [float(x[1]) for x in lst]
y = [float(x[2]) for x in lst]

In [6]:
#divide identification set and validation set
ID_LENGHT = 3000
time_id = time[:ID_LENGHT]
u_id = u[:ID_LENGHT]
y_id = y[:ID_LENGHT]

time_val = time[ID_LENGHT:]
u_val = u[ID_LENGHT:]
y_val = y[ID_LENGHT:]

In [11]:
from sknn.mlp import Regressor , Layer

nn = Regressor(
        layers=[
            Layer("Sigmoid", units = 14),
            Layer("Sigmoid", units = 7),
            Layer("Linear")],
        learning_rate=0.02,
        n_iter = 10)

In [8]:
Y = np.array(y_id[10:len(y_id)])
PHI = []
for i in tqdm(range(ID_LENGHT - 10)):
    regressors = [u_id[i:i+10] + y_id[6+i:10+i]]
    PHI.append(regressors[0])
PHI = np.array(PHI)

100%|██████████████████████████████████████████████████████████████████████████| 2990/2990 [00:00<00:00, 373855.09it/s]


In [12]:
nn.fit(PHI,Y)

ImportError: cannot import name 'downsample'

In [ ]:
%pylab inline
pylab.rcParams['figure.figsize'] = (20, 10)

plt.subplot(211)
plt.plot(y_hat[:2990], color='blue')
plt.subplot(212)
plt.plot(y_id[10:3000], color='red')
plt.show()

In [ ]:
plt.plot(y_id[10:3000], color='red')
plt.plot(y_hat[:2990], color='blue')
plt.show()